**Install the necessities**

In [2]:
# !pip -q install git+https://github.com/huggingface/transformers
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain
# !pip install flask
!pip install pdfplumber
!pip install pytesseract
!pip install pdf2image
!apt-get install tesseract-ocr
!apt-get install -y poppler-utils
!pip uninstall -y pyarrow requests
!pip install pyarrow==14.0.1 requests==2.31.0 transformers datasets torch
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 836.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 84.0 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 0 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe

**Random buggo**

In [ ]:
pip install flask --ignore-installed blinker

  Using cached flask-3.0.3-py3-none-any.whl (101 kB)
  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 10.0 MB/s eta 0:00:00
  Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.9 MB/s eta 0:00:00


**Drive Access**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing libs**

In [3]:
# from flask import Flask
# from threading import Thread
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score
import time

**Text Extraction using pdfPlumber and PyTesseract**

OCR based extraction for unreadable txts

In [4]:
def ocr_execution(file_path,i):
  pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

  def clean_ocr_text(text):
      cleaned_text = re.sub(r'\(cid:\d+\)', '', text)
      cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
      return cleaned_text

  def extract_text_from_pdf_page_using_ocr(file_path, page_number):
      pages = convert_from_path(file_path, first_page=page_number, last_page=page_number)
      if not pages:
        return ""
      page_image = pages[0]
      ocr_text = pytesseract.image_to_string(page_image)
      if ocr_text:
          cleaned_text = clean_ocr_text(ocr_text)
          return cleaned_text
      else:
          return ""

  page_number = i
  extracted_text = extract_text_from_pdf_page_using_ocr(file_path, page_number)
  exttext = []
  words = extracted_text.split()
  exttext = [word for word in words if word]
  return exttext

Basic extraction

In [5]:
def extracter(file_path):
  doc = pdfplumber.open(file_path)
  text = []
  extext = []
  print('---------------------------------------------------------------',file_path,'----------------------------------------------------------------')
  for i in doc.pages:
      table = i.extract_table()
      if 'cid' not in table[0][0]:
        print(table)
        text.extend(table)
      else:
        ocr_text = ocr_execution(file_path,i.page_number)
        print(ocr_text)
        text.append(ocr_text)
  for row in text:
      for cell in row:
          if cell not in [None, '']:
              extext.append(cell)
  print()
  print(len(extext))
  print(extext)
  print()

Directory Initializer

In [6]:
directory_path = '/content/drive/MyDrive/mr_files'
directory_files = os.listdir(directory_path)

Printing the stuff extracted

In [ ]:
for files in directory_files:
  file_path = os.path.join(directory_path, files)
  extracter(file_path)

--------------------------------------------------------------- /content/drive/MyDrive/mr_files/498606-mr.pdf ----------------------------------------------------------------
['eat', 'ORR', 'AUT', 'CRATE', 'Tt', 'FUT', '(A', 'Govt', 'of', 'india', 'Undertaking)', 'PUMP', 'PROCESS', 'DATA', 'SHEET', 'PROJECT', '_', 'REPLACEMENT', 'AND', 'ADDITIONAL', 'INSTALLATION', 'OF', 'COMP', 'CLIENT', 'ONGC', 'Document', 'No.', 'B650-020-02-42-DS-1603', 'Rev.No.', '4', 'Page', '1', 'of', '2', 'UNIT', 'ITEM', 'NO.', 'Replacement', 'of', 'CSU', 'Off', 'Gas', 'Compressors', '&', 'One', 'Flare', 'KOC', 'JOB', 'NO.', 'B650', '020-P-611A/B', 'UNITNO.', '020', 'DESIGN', 'CASE', 'SERVICE', 'RD', 'STAGE', 'HC', 'CONDENSATE', 'PUMPS.', 'PARALLEL', 'OPERATION', 'NO', 'OPERATION', 'INTERMITTENT', 'OPERATING', '1', 'STAND', 'BY', '1', 'TYPE', 'OF', 'PUMP.', 'CENTRIFUGAL', 'PROPERTIES', 'OF', 'LIQUID', 'LIQUID', 'HANDLED', 'HYDROCARBON', 'CONDENSATE', 'PUMPING', 'TEMPERATURE', 'degC', '|', '10-60', 'VISCOSITY', 

Printing extracted stuff without noise

In [7]:
def extracter(file_path):
  doc = pdfplumber.open(file_path)
  text = []
  extext = []
  for i in doc.pages:
      table = i.extract_table()
      if 'cid' not in table[0][0]:
        text.extend(table)
      else:
        ocr_text = ocr_execution(file_path,i.page_number)
        text.append(ocr_text)
  for row in text:
      for cell in row:
          if cell not in [None, '']:
              extext.append(cell)
  return extext

Dictionary for each MR component and to which it belongs

In [8]:
data = {}
for files in directory_files:
  file_path = os.path.join(directory_path, files)
  data[files] = extracter(file_path)

print(len(data))
print(data)

8
{'498606-mr.pdf': ['eat', 'ORR', 'AUT', 'CRATE', 'Tt', 'FUT', '(A', 'Govt', 'of', 'india', 'Undertaking)', 'PUMP', 'PROCESS', 'DATA', 'SHEET', 'PROJECT', '_', 'REPLACEMENT', 'AND', 'ADDITIONAL', 'INSTALLATION', 'OF', 'COMP', 'CLIENT', 'ONGC', 'Document', 'No.', 'B650-020-02-42-DS-1603', 'Rev.No.', '4', 'Page', '1', 'of', '2', 'UNIT', 'ITEM', 'NO.', 'Replacement', 'of', 'CSU', 'Off', 'Gas', 'Compressors', '&', 'One', 'Flare', 'KOC', 'JOB', 'NO.', 'B650', '020-P-611A/B', 'UNITNO.', '020', 'DESIGN', 'CASE', 'SERVICE', 'RD', 'STAGE', 'HC', 'CONDENSATE', 'PUMPS.', 'PARALLEL', 'OPERATION', 'NO', 'OPERATION', 'INTERMITTENT', 'OPERATING', '1', 'STAND', 'BY', '1', 'TYPE', 'OF', 'PUMP.', 'CENTRIFUGAL', 'PROPERTIES', 'OF', 'LIQUID', 'LIQUID', 'HANDLED', 'HYDROCARBON', 'CONDENSATE', 'PUMPING', 'TEMPERATURE', 'degC', '|', '10-60', 'VISCOSITY', 'AT', 'PUMPING', 'TEMPERATURE', 'CST', '|', '0.15', 'VAPOUR', 'PRESSURE', 'AT', 'PUMPING', 'TEMPERATURE', 'KG/CM2', 'A', '49.012', 'LIQUID', 'DENSITY', 'AT

Label prep

In [9]:
def alphanumeric_to_numeric(alpha_string):
  alphanumeric_string = alpha_string
  numeric_part = re.search(r'\d+', alphanumeric_string).group()
  numeric_integer = int(numeric_part)
  return numeric_integer

Separating the labels and features

In [10]:
comps = []
labels = []
for label, elements in data.items():
    for element in elements:
        comps.append(element)
        labels.append(alphanumeric_to_numeric(label))

print(labels)
print(comps)
print(len(comps))
print(len(labels))

[498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606, 498606,

Making train/test labels/texts

In [12]:
train_texts = []
test_texts = []
train_labels = []
test_labels = []

#80% training and remaining test
# e = (80/100)*len(data)
# e = int(e)

temp = list(data.items())
i = 0
while (i<len(data)):
  if(i != 3 or i != 4 or i != 5):
    train_texts += temp[i][1]
    for j in range(len(temp[i][1])):
      train_labels.append(alphanumeric_to_numeric(temp[i][0]))
  i+=1

test_labels.append(alphanumeric_to_numeric(temp[3][0]))
test_texts.append(temp[3][1])
test_labels.append(alphanumeric_to_numeric(temp[4][0]))
test_texts.append(temp[4][1])
test_labels.append(alphanumeric_to_numeric(temp[5][0]))
test_texts.append(temp[5][1])

In [13]:
print(train_texts)
print(train_labels)
print(test_texts)
print(test_labels)

['eat', 'ORR', 'AUT', 'CRATE', 'Tt', 'FUT', '(A', 'Govt', 'of', 'india', 'Undertaking)', 'PUMP', 'PROCESS', 'DATA', 'SHEET', 'PROJECT', '_', 'REPLACEMENT', 'AND', 'ADDITIONAL', 'INSTALLATION', 'OF', 'COMP', 'CLIENT', 'ONGC', 'Document', 'No.', 'B650-020-02-42-DS-1603', 'Rev.No.', '4', 'Page', '1', 'of', '2', 'UNIT', 'ITEM', 'NO.', 'Replacement', 'of', 'CSU', 'Off', 'Gas', 'Compressors', '&', 'One', 'Flare', 'KOC', 'JOB', 'NO.', 'B650', '020-P-611A/B', 'UNITNO.', '020', 'DESIGN', 'CASE', 'SERVICE', 'RD', 'STAGE', 'HC', 'CONDENSATE', 'PUMPS.', 'PARALLEL', 'OPERATION', 'NO', 'OPERATION', 'INTERMITTENT', 'OPERATING', '1', 'STAND', 'BY', '1', 'TYPE', 'OF', 'PUMP.', 'CENTRIFUGAL', 'PROPERTIES', 'OF', 'LIQUID', 'LIQUID', 'HANDLED', 'HYDROCARBON', 'CONDENSATE', 'PUMPING', 'TEMPERATURE', 'degC', '|', '10-60', 'VISCOSITY', 'AT', 'PUMPING', 'TEMPERATURE', 'CST', '|', '0.15', 'VAPOUR', 'PRESSURE', 'AT', 'PUMPING', 'TEMPERATURE', 'KG/CM2', 'A', '49.012', 'LIQUID', 'DENSITY', 'AT', 'PUMPING', 'TEMPE

Data prep

In [14]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

Building/Initializing model

In [15]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_classes, vocab_size, hidden_dim=768, num_layers=6, num_heads=8, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)
        transformer_output = self.transformer_encoder(embeddings)
        pooled_output = transformer_output[:, 0, :]
        output = self.dropout(pooled_output)
        return self.fc(output)

Training model

In [16]:
def train(model, train_loader, optimizer, criterion, device, epochs=3, gradient_accumulation_steps=4):
    model.train()
    scaler = GradScaler()
    for epoch in range(epochs):
        total_loss = 0
        start_time = time.time()
        for step, batch in enumerate(train_loader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            with autocast():
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                loss = loss / gradient_accumulation_steps
            scaler.scale(loss).backward()
            if (step + 1) % gradient_accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            total_loss += loss.item() * gradient_accumulation_steps
            if (step + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Step {step + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch + 1} finished in {epoch_time:.2f} seconds with average loss: {total_loss / len(train_loader):.4f}")

Similarity Index

In [21]:
def sim_index(model, tokenizer, device, text, top_k=3):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = nn.functional.softmax(outputs, dim=1)
        top_probs, top_labels = torch.topk(probs, top_k, dim=1)
    top_labels = top_labels.cpu().numpy()[0]
    top_probs = top_probs.cpu().numpy()[0]
    return [(unique_labels[label], prob) for label, prob in zip(top_labels, top_probs)]

Model evaluation

In [18]:
def evaluate(model, test_loader, device):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f'Accuracy: {accuracy:.4f}')

Training begins here....

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128
unique_labels = list(set(train_labels + test_labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
labels_train = [label_map[label] for label in train_labels]
labels_test = [label_map[label] for label in test_labels]
train_dataset = TextDataset(train_texts, labels_train, tokenizer, max_len)
test_dataset = TextDataset(test_texts, labels_test, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(unique_labels)
vocab_size = tokenizer.vocab_size
model = TransformerClassifier(num_classes=num_classes, vocab_size=vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

train(model, train_loader, optimizer, criterion, device, epochs=3, gradient_accumulation_steps=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/3, Step 10/457, Loss: 0.5306
Epoch 1/3, Step 20/457, Loss: 0.5226
Epoch 1/3, Step 30/457, Loss: 0.5774
Epoch 1/3, Step 40/457, Loss: 0.6040
Epoch 1/3, Step 50/457, Loss: 0.5778
Epoch 1/3, Step 60/457, Loss: 0.5072
Epoch 1/3, Step 70/457, Loss: 0.5487
Epoch 1/3, Step 80/457, Loss: 0.5418
Epoch 1/3, Step 90/457, Loss: 0.4581
Epoch 1/3, Step 100/457, Loss: 0.5038
Epoch 1/3, Step 110/457, Loss: 0.5539
Epoch 1/3, Step 120/457, Loss: 0.5328
Epoch 1/3, Step 130/457, Loss: 0.5699
Epoch 1/3, Step 140/457, Loss: 0.5385
Epoch 1/3, Step 150/457, Loss: 0.5409
Epoch 1/3, Step 160/457, Loss: 0.5496
Epoch 1/3, Step 170/457, Loss: 0.5388
Epoch 1/3, Step 180/457, Loss: 0.5513
Epoch 1/3, Step 190/457, Loss: 0.5517
Epoch 1/3, Step 200/457, Loss: 0.5986
Epoch 1/3, Step 210/457, Loss: 0.5496
Epoch 1/3, Step 220/457, Loss: 0.5789
Epoch 1/3, Step 230/457, Loss: 0.5215
Epoch 1/3, Step 240/457, Loss: 0.5351
Epoch 1/3, Step 250/457, Loss: 0.5206
Epoch 1/3, Step 260/457, Loss: 0.5355
Epoch 1/3, Step 270/4

In [20]:
print(labels_train)
print(labels_test)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

**User Inputs here...**

In [28]:
user_input = "PUMP with PUMPING TEMPERATURE 5-40"
labels_prob = sim_index(model, tokenizer, device, user_input, top_k=3)
print(f"Top 3 predicted labels for user input: {labels_prob}")

Top 3 predicted labels for user input: [(498108, 0.17159855), (496838, 0.16281648), (498645, 0.15894982)]
